# CORA - Categorizing academic publications using getML

In this notebook, we compare getML against extant approaches in the relational learning literature on the CORA data set, which is often used for benchmarking. We demonstrate that getML outperforms the state of the art in the relational learning literature on this data set. Beyond the benchmarking aspects, this notebooks showcases getML's excellent capabilities in dealing with categorical data.

Summary:

- Prediction type: __Classification model__
- Domain: __Academia__
- Prediction target: __The category of a paper__ 
- Population size: __2708__

_Author: Dr. Patrick Urbanke_

# Background

CORA is a well-known benchmarking dataset in the academic literature on relational learning. The dataset contains 2708 scientific publications on machine learning. The papers are divided into 7 categories. The challenge is to predict the category of a paper based on the papers it cites, the papers it is cited by and keywords contained in the paper.

It has been downloaded from the [CTU Prague relational learning repository](https://relational.fit.cvut.cz/dataset/CORA) (Motl and Schulte, 2015).

### A web frontend for getML

The getML monitor is a frontend built to support your work with getML. The getML monitor displays information such as the imported data frames, trained pipelines and allows easy data and feature exploration. You can launch the getML monitor [here](http://localhost:1709).

### Where is this running?

Your getML live session is running inside a docker container on [mybinder.org](https://mybinder.org/), a service built by the Jupyter community and funded by Google Cloud, OVH, GESIS Notebooks and the Turing Institute. As it is a free service, this session will shut down after 10 minutes of inactivity.

# Analysis

Let's get started with the analysis and set up your session:

In [1]:
import copy
import os
from urllib import request

import numpy as np
import pandas as pd
from IPython.display import Image
import matplotlib.pyplot as plt
plt.style.use('seaborn')
%matplotlib inline  

import getml

getml.engine.set_project('cora')


Connected to project 'cora'


## 1. Loading data

### 1.1 Download from source

We begin by downloading the data from the source file:

In [2]:
conn = getml.database.connect_mariadb(
    host="relational.fit.cvut.cz",
    dbname="CORA",
    port=3306,
    user="guest",
    password="relational"
)

conn

Connection(conn_id='default',
           dbname='CORA',
           dialect='mysql',
           host='relational.fit.cvut.cz',
           port=3306)

In [3]:
def load_if_needed(name):
    """
    Loads the data from the relational learning
    repository, if the data frame has not already
    been loaded.
    """
    if not getml.data.exists(name):
        data_frame = getml.data.DataFrame.from_db(
            name=name,
            table_name=name,
            conn=conn
        )
        data_frame.save()
    else:
        data_frame = getml.data.load_data_frame(name)
    return data_frame

In [4]:
paper = load_if_needed("paper")
cites = load_if_needed("cites")
content = load_if_needed("content")

In [5]:
paper

Name,paper_id,class_label
Role,join_key,categorical
0,35,Genetic_Algorithms
1,40,Genetic_Algorithms
2,114,Reinforcement_Learning
3,117,Reinforcement_Learning
4,128,Reinforcement_Learning
,...,...
2703,1154500,Case_Based
2704,1154520,Neural_Networks
2705,1154524,Rule_Learning


In [6]:
cites

Name,cited_paper_id,citing_paper_id
Role,join_key,join_key
0,35,887
1,35,1033
2,35,1688
3,35,1956
4,35,8865
,...,...
5424,853116,19621
5425,853116,853155
5426,853118,1140289


In [7]:
content

Name,paper_id,word_cited_id
Role,join_key,categorical
0,35,word100
1,35,word1152
2,35,word1175
3,35,word1228
4,35,word1248
,...,...
49211,1155073,word75
49212,1155073,word759
49213,1155073,word789


### 1.2 Prepare data for getML

getML requires that we define *roles* for each of the columns.

In [8]:
paper.set_role("paper_id", getml.data.roles.join_key)
paper.set_role("class_label", getml.data.roles.categorical)
paper

Name,paper_id,class_label
Role,join_key,categorical
0,35,Genetic_Algorithms
1,40,Genetic_Algorithms
2,114,Reinforcement_Learning
3,117,Reinforcement_Learning
4,128,Reinforcement_Learning
,...,...
2703,1154500,Case_Based
2704,1154520,Neural_Networks
2705,1154524,Rule_Learning


In [9]:
cites.set_role(["cited_paper_id", "citing_paper_id"], getml.data.roles.join_key)
cites

Name,cited_paper_id,citing_paper_id
Role,join_key,join_key
0,35,887
1,35,1033
2,35,1688
3,35,1956
4,35,8865
,...,...
5424,853116,19621
5425,853116,853155
5426,853118,1140289


We need to separate our data set into a training, testing and validation set:

In [10]:
content.set_role("paper_id", getml.data.roles.join_key)
content.set_role("word_cited_id", getml.data.roles.categorical)
content

Name,paper_id,word_cited_id
Role,join_key,categorical
0,35,word100
1,35,word1152
2,35,word1175
3,35,word1228
4,35,word1248
,...,...
49211,1155073,word75
49212,1155073,word759
49213,1155073,word789


We also have to separate the data set into a training and testing set:

In [11]:
split = getml.data.split.random(train=0.7, test=0.3, validation=0.0)

data_train = paper[split=="train"].to_df("data_train")
data_test = paper[split=="test"].to_df("data_test")

In [12]:
data_train

Name,paper_id,class_label
Role,join_key,categorical
0,35,Genetic_Algorithms
1,114,Reinforcement_Learning
2,130,Reinforcement_Learning
3,288,Reinforcement_Learning
4,424,Rule_Learning
,...,...
1882,1154276,Neural_Networks
1883,1154459,Genetic_Algorithms
1884,1154500,Case_Based


The goal is to predict seven different labels. We generate a target column for each of those labels:

In [13]:
class_label = paper.class_label.unique()
class_label

array(['Case_Based', 'Genetic_Algorithms', 'Neural_Networks',
       'Probabilistic_Methods', 'Reinforcement_Learning', 'Rule_Learning',
       'Theory'], dtype='<U22')

In [14]:
def make_target_columns(data_frame):
    for label in class_label:
        name = "class_label=" + label
        data_frame[name] = (data_frame.class_label == label)
        data_frame.set_role(name, getml.data.roles.target)
    data_frame.set_role("class_label", getml.data.roles.unused_string)

In [15]:
make_target_columns(data_train)
make_target_columns(data_test)

In [16]:
data_train

Name,paper_id,class_label=Case_Based,class_label=Genetic_Algorithms,class_label=Neural_Networks,class_label=Probabilistic_Methods,class_label=Reinforcement_Learning,class_label=Rule_Learning,class_label=Theory,class_label
Role,join_key,target,target,target,target,target,target,target,unused_string
0,35,0,1,0,0,0,0,0,Genetic_Algorithms
1,114,0,0,0,0,1,0,0,Reinforcement_Learning
2,130,0,0,0,0,1,0,0,Reinforcement_Learning
3,288,0,0,0,0,1,0,0,Reinforcement_Learning
4,424,0,0,0,0,0,1,0,Rule_Learning
,...,...,...,...,...,...,...,...,...
1882,1154276,0,0,1,0,0,0,0,Neural_Networks
1883,1154459,0,1,0,0,0,0,0,Genetic_Algorithms
1884,1154500,1,0,0,0,0,0,0,Case_Based


In [17]:
data_test

Name,paper_id,class_label=Case_Based,class_label=Genetic_Algorithms,class_label=Neural_Networks,class_label=Probabilistic_Methods,class_label=Reinforcement_Learning,class_label=Rule_Learning,class_label=Theory,class_label
Role,join_key,target,target,target,target,target,target,target,unused_string
0,40,0,1,0,0,0,0,0,Genetic_Algorithms
1,117,0,0,0,0,1,0,0,Reinforcement_Learning
2,128,0,0,0,0,1,0,0,Reinforcement_Learning
3,164,0,0,0,0,0,0,1,Theory
4,463,1,0,0,0,0,0,0,Case_Based
,...,...,...,...,...,...,...,...,...
816,1154173,0,0,0,1,0,0,0,Probabilistic_Methods
817,1154229,0,0,0,1,0,0,0,Probabilistic_Methods
818,1154233,0,0,0,1,0,0,0,Probabilistic_Methods


In [18]:
container = getml.next.DataContainer(train=data_train, test=data_test)
container.add(paper=paper, cites=cites, content=content)
container.freeze()

## 2. Predictive modeling

We loaded the data and defined the roles and units. Next, we create a getML pipeline for relational learning.

### 2.1 Define relational model

To get started with relational learning, we need to specify the data model. Even though the data set itself is quite simple with only three tables and six columns in total, the resulting data model is actually quite complicated.

That is because the class label can be predicting using three different pieces of information:

- The keywords used by the paper
- The keywords used by papers it cites and by papers that cite the paper
- The class label of papers it cites and by papers that cite the paper

In [19]:
dm = getml.next.DataModel(paper.to_placeholder())

dm.add(getml.next.to_placeholder(cites, cites, content=content, paper=paper))

dm.population.join(
    dm.cites[0],
    on=('paper_id', 'cited_paper_id')
)

dm.cites[0].join(
    dm.content,
    on=('citing_paper_id', 'paper_id')
)

dm.cites[0].join(
    dm.paper,
    on=('citing_paper_id', 'paper_id'),
    cardinality=getml.data.relationship.many_to_one
)

dm.population.join(
    dm.cites[1],
    on=('paper_id', 'citing_paper_id')
)

dm.cites[1].join(
    dm.content,
    on=('cited_paper_id', 'paper_id')
)

dm.cites[1].join(
    dm.paper,
    on=('cited_paper_id', 'paper_id'),
    cardinality=getml.data.relationship.many_to_one
)

dm.population.join(
    dm.content,
    on='paper_id'
)

dm

placeholder   other placeholder   allow lagged targets   horizon   join keys used    memory   other join keys used   other time stamps used   relationship   time stamps used   upper time stamps used
0   paper         cites               False                  0.0       paper_id          0.0      cited_paper_id                                  many-to-many                                            
1   paper         cites               False                  0.0       paper_id          0.0      citing_paper_id                                 many-to-many                                            
2   paper         content             False                  0.0       paper_id          0.0      paper_id                                        many-to-many                                            
3   cites         content             False                  0.0       citing_paper_id   0.0      paper_id                                        many-to-many                                            
4   cites         paper               False                  0.0       citing_paper_id   0.0      paper_id                                        many-to-one                                             
5   cites         content             False                  0.0       cited_paper_id    0.0      paper_id                                        many-to-many                                            
6   cites         paper               False                  0.0       cited_paper_id    0.0      paper_id                                        many-to-one

### 2.2 getML pipeline

<!-- #### 2.1.1  -->
__Set-up the feature learner & predictor__

We use the relboost algorithms for this problem. Because of the large number of keywords, we regularize the model a bit by requiring a minimum support for the keywords (`min_num_samples`).

In [20]:
mapping = getml.preprocessors.Mapping()

fast_prop = getml.feature_learning.FastPropModel(
    loss_function=getml.feature_learning.loss_functions.CrossEntropyLoss,
    num_threads=1
)

relboost = getml.feature_learning.RelboostModel(
    num_features=10,
    num_subfeatures=10,
    loss_function=getml.feature_learning.loss_functions.CrossEntropyLoss,
    seed=4367,
    num_threads=1,
    min_num_samples=30
)

predictor = getml.predictors.XGBoostClassifier()

__Build the pipeline__

In [21]:
pipe1 = getml.pipeline.Pipeline(
    tags=['fast_prop'],
    population=dm,
    preprocessors=[mapping],
    feature_learners=[fast_prop],
    predictors=[predictor]
)

pipe1

Pipeline(preprocessors=['Mapping'],
         feature_learners=['FastPropModel'],
         feature_selectors=[],
         include_categorical=False,
         peripheral=['cites', 'content', 'paper'],
         population='paper',
         predictors=['XGBoostClassifier'],
         tags=['fast_prop'],
         share_selected_features=0.5)

In [22]:
pipe2 = getml.pipeline.Pipeline(
    tags=['relboost'],
    population=dm,
    feature_learners=[relboost],
    predictors=[predictor]
)

pipe2

Pipeline(preprocessors=[],
         feature_learners=['RelboostModel'],
         feature_selectors=[],
         include_categorical=False,
         peripheral=['cites', 'content', 'paper'],
         population='paper',
         predictors=['XGBoostClassifier'],
         tags=['relboost'],
         share_selected_features=0.5)

### 2.3 Model training

In [23]:
pipe1.check(container.train)

Checking data model...

Preprocessing...
[========================================] 100%

INFO [MIGHT TAKE LONG]: The number of unique entries in column 'word_cited_id' in CONTENT__STAGING_TABLE_4 is 1432. This might take a long time to fit. You should consider setting its role to unused_string or using it for comparison only (you can do the latter by setting a unit that contains 'comparison only').
INFO [FOREIGN KEYS NOT FOUND]: When joining PAPER__STAGING_TABLE_1 and CITES__STAGING_TABLE_2 over 'paper_id' and 'cited_paper_id', there are no corresponding entries for 41.759406% of entries in 'paper_id' in 'PAPER__STAGING_TABLE_1'. You might want to double-check your join keys.
INFO [FOREIGN KEYS NOT FOUND]: When joining PAPER__STAGING_TABLE_1 and CITES__STAGING_TABLE_3 over 'paper_id' and 'citing_paper_id', there are no corresponding entries for 17.700053% of entries in 'paper_id' in 'PAPER__STAGING_TABLE_1'. You might want to double-check your join keys.


In [24]:
pipe1.fit(container.train)

Checking data model...

INFO [MIGHT TAKE LONG]: The number of unique entries in column 'word_cited_id' in CONTENT__STAGING_TABLE_4 is 1432. This might take a long time to fit. You should consider setting its role to unused_string or using it for comparison only (you can do the latter by setting a unit that contains 'comparison only').
INFO [FOREIGN KEYS NOT FOUND]: When joining PAPER__STAGING_TABLE_1 and CITES__STAGING_TABLE_2 over 'paper_id' and 'cited_paper_id', there are no corresponding entries for 41.759406% of entries in 'paper_id' in 'PAPER__STAGING_TABLE_1'. You might want to double-check your join keys.
INFO [FOREIGN KEYS NOT FOUND]: When joining PAPER__STAGING_TABLE_1 and CITES__STAGING_TABLE_3 over 'paper_id' and 'citing_paper_id', there are no corresponding entries for 17.700053% of entries in 'paper_id' in 'PAPER__STAGING_TABLE_1'. You might want to double-check your join keys.

Preprocessing...
[========================================] 100%

FastProp: Trying 3780 feature

Pipeline(preprocessors=['Mapping'],
         feature_learners=['FastPropModel'],
         feature_selectors=[],
         include_categorical=False,
         peripheral=['cites', 'content', 'paper'],
         population='paper',
         predictors=['XGBoostClassifier'],
         tags=['fast_prop', 'container-50Ehaf'],
         share_selected_features=0.5)

url: http://localhost:1709/#/getpipeline/cora/MYMaoh/0/

In [25]:
pipe2.check(container.train)

Checking data model...

INFO [MIGHT TAKE LONG]: The number of unique entries in column 'word_cited_id' in CONTENT__STAGING_TABLE_4 is 1432. This might take a long time to fit. You should consider setting its role to unused_string or using it for comparison only (you can do the latter by setting a unit that contains 'comparison only').
INFO [FOREIGN KEYS NOT FOUND]: When joining PAPER__STAGING_TABLE_1 and CITES__STAGING_TABLE_2 over 'paper_id' and 'cited_paper_id', there are no corresponding entries for 41.759406% of entries in 'paper_id' in 'PAPER__STAGING_TABLE_1'. You might want to double-check your join keys.
INFO [FOREIGN KEYS NOT FOUND]: When joining PAPER__STAGING_TABLE_1 and CITES__STAGING_TABLE_3 over 'paper_id' and 'citing_paper_id', there are no corresponding entries for 17.700053% of entries in 'paper_id' in 'PAPER__STAGING_TABLE_1'. You might want to double-check your join keys.


The training process seems a bit intimidating. That is because the relboost algorithms needs to train separate models for each class label. This is due to the nature of the generated features.

In [ ]:
pipe2.fit(container.train)

Checking data model...

INFO [MIGHT TAKE LONG]: The number of unique entries in column 'word_cited_id' in CONTENT__STAGING_TABLE_4 is 1432. This might take a long time to fit. You should consider setting its role to unused_string or using it for comparison only (you can do the latter by setting a unit that contains 'comparison only').
INFO [FOREIGN KEYS NOT FOUND]: When joining PAPER__STAGING_TABLE_1 and CITES__STAGING_TABLE_2 over 'paper_id' and 'cited_paper_id', there are no corresponding entries for 41.759406% of entries in 'paper_id' in 'PAPER__STAGING_TABLE_1'. You might want to double-check your join keys.
INFO [FOREIGN KEYS NOT FOUND]: When joining PAPER__STAGING_TABLE_1 and CITES__STAGING_TABLE_3 over 'paper_id' and 'citing_paper_id', there are no corresponding entries for 17.700053% of entries in 'paper_id' in 'PAPER__STAGING_TABLE_1'. You might want to double-check your join keys.

Relboost: Training subfeatures...
[========================================] 100%

Relboost: Tr

### 2.4 Model evaluation

In [ ]:
pipe1.score(container.test)

In [ ]:
pipe2.score(container.test)

To make things a bit easier, we just look at our test results.

In [ ]:
pipe1.scores.filter(lambda score: score.set_used == "data_test")

In [ ]:
pipe2.scores.filter(lambda score: score.set_used == "data_test")

We take the average of the AUC values, which is also the value that appears in the getML monitor (http://localhost:1709/#/listpipelines/cora).

In [ ]:
print(np.mean(pipe1.auc))
print(np.mean(pipe2.auc))

The accuracy for multiple targets can be calculated using one of two methods. The first method is to simply take the average of the pair-wise accuracy values, which is also the value that appears in the getML monitor (http://localhost:1709/#/listpipelines/cora).

In [ ]:
print(np.mean(pipe1.accuracy))
print(np.mean(pipe2.accuracy))

However, the benchmarking papers actually use a different approach: 

- They first generate probabilities for each of the labels:

In [ ]:
probabilities1 = pipe1.predict(container.test)
probabilities2 = pipe2.predict(container.test)

- They then find the class label with the highest probability:

In [ ]:
ix_max = np.argmax(probabilities1, axis=1)
predicted_labels1 = np.asarray([class_label[ix] for ix in ix_max])

ix_max = np.argmax(probabilities2, axis=1)
predicted_labels2 = np.asarray([class_label[ix] for ix in ix_max])

- They then compare that value to the actual class label:

In [ ]:
actual_labels = data_test.class_label.to_numpy()

print("Share of accurately predicted class labels (pipe1):")
print((actual_labels == predicted_labels1).sum() / len(actual_labels))
print()
print("Share of accurately predicted class labels (pipe2):")
print((actual_labels == predicted_labels2).sum() / len(actual_labels))
print()

Since this is the method the benchmark papers use, this is the accuracy score we will report as well.

### 2.6 Studying features

__Feature correlations__

We want to analyze how the features are correlated with the target variables.

In [ ]:
TARGET_NUM = 0

In [ ]:
names, correlations = pipe2.features.correlations(target_num=TARGET_NUM)

plt.subplots(figsize=(20, 10))

plt.bar(names, correlations, color='#6829c2')

plt.title('Feature correlations with class label ' + class_label[TARGET_NUM])
plt.xlabel('Features')
plt.ylabel('Correlations')
plt.xticks(rotation='vertical')
plt.show()

__Feature importances__
 
Feature importances are calculated by analyzing the improvement in predictive accuracy on each node of the trees in the XGBoost predictor. They are then normalized, so that all importances add up to 100%.

In [ ]:
names, importances = pipe2.features.importances()

plt.subplots(figsize=(20, 10))

plt.bar(names, importances, color='#6829c2')

plt.title('Feature importances for class label ' + class_label[TARGET_NUM])
plt.xlabel('Features')
plt.ylabel('Importances')
plt.xticks(rotation='vertical')
plt.show()

__Column importances__

Because getML uses relational learning, we can apply the principles we used to calculate the feature importances to individual columns as well.

In [ ]:
names, importances = pipe2.columns.importances(target_num=TARGET_NUM)

plt.subplots(figsize=(20, 10))

plt.bar(names, importances, color='#6829c2')

plt.title('Columns importances for class label ' + class_label[TARGET_NUM])
plt.xlabel('Columns')
plt.ylabel('Importances')
plt.xticks(rotation='vertical')
plt.show()

__Transpiling the learned features__

We can also transpile the learned features to SQLite3 code. This is useful for productionization. Please also refer to the `sqlite3` module in the getML Python API.

In [ ]:
pipe1.features.to_sql().save("cora", split=True)

### 2.7 Benchmarks

State-of-the-art approaches on this data set perform as follows:

| Approach                      | Study                      | Accuracy     | AUC     |
| :--------------------------   | :------------------------  | -----------: | ------: |
| RelF                          | Dinh et al (2012)          | 85.7%        |     --  |
| LBP                           | Dinh et al (2012)          | 85.0%        |     --  |
| EPRN                          | Preisach and Thieme (2006) | 84.0%        |     --  |
| PRN                           | Preisach and Thieme (2006) | 81.0%        |     --  |
| ACORA                         | Perlich and Provost (2006) | --           |   97.0% |


As we can see, the performance of the relboost algorithm, as used in this notebook, compares favorably to these benchmarks.

| Approach                      | Accuracy     | AUC     |
| :---------------------------- | -----------: | ------: |
| FastProp                      | 89.9%        | 98.5%   |
| Relboost                      | 89.9%        | 98.3%   |


## 3. Conclusion

In this notebook we have demonstrated that getML outperforms state-of-the-art relational learning algorithms on the CORA dataset.

## References

Dinh, Quang-Thang, Christel Vrain, and Matthieu Exbrayat. "A Link-Based Method for Propositionalization." ILP (Late Breaking Papers). 2012.

Motl, Jan, and Oliver Schulte. "The CTU prague relational learning repository." arXiv preprint arXiv:1511.03086 (2015).

Perlich, Claudia, and Foster Provost. "Distribution-based aggregation for relational learning with identifier attributes." Machine Learning 62.1-2 (2006): 65-105.

Preisach, Christine, and Lars Schmidt-Thieme. "Relational ensemble classification." Sixth International Conference on Data Mining (ICDM'06). IEEE, 2006.

# Next Steps

This tutorial benchmarked getML against academic state-of-the-art algorithms from relational learning literature and getML's qualities with respect to categorical data.

If you are interested in further real-world applications of getML, head back to the [notebook overview](welcome.md) and choose one of the remaining examples.

Here is some additional material from our [documentation](https://docs.getml.com/latest/) if you want to learn more about getML:
* [Feature learning with Multirel](https://docs.getml.com/latest/user_guide/feature_engineering/feature_engineering.html#multirel)
* [Feature learning with Relboost](https://docs.getml.com/latest/user_guide/feature_engineering/feature_engineering.html#relboost)

# Get in contact

If you have any question schedule a [call with Alex](https://go.getml.com/meetings/alexander-uhlig/getml-demo), the co-founder of getML, or write us an [email](team@getml.com). Prefer a private demo of getML? Just contact us to make an appointment.